In [118]:
import numpy as np

sample = """
_______________________
_5_6-$_________________
_|/____________________
_*-2___________________
_|_____________________
_$_____________________
_______________________
_______________________
_______________________
"""

directions = {
    '|': [0, 1],
    '/': [-1, 1],
    '-': [1, 0]
}

initial = {
    '*': 1,
    '+': 0,
    '$': 0
}

def int_(val):
    try:
        return int(val)
    except:
        return 0

def execute(program, i=3):
    lines = [list(l) for l in program.split('\n')[1:-1]]
    ops = '+*$'
    lines = np.array(lines)
#     size = len(lines[0]), len(lines)
    size = lines.shape
    print(lines.shape)
    t = tuple
#     new = ['_' * size[0]] * size[1]
#     new = np.full(lines.shape, '_', dtype=object)

    for iteration in range(i):
        new = lines.copy().astype(object)
        for x, y in np.ndindex(lines.shape):
            c = lines[x][y]
            if c in ops and False:
    #             new[x][y] = 0
                if c in initial:
                    n = initial[c]
                else:
                    n = 0
                new[x][y] = n

        for x, y in np.ndindex(lines.shape):
            w = np.array([x, y])
    #         w = np.flip(w)
            c = lines[x][y]
            if c in directions:
                d = directions[c]
                d = np.flip(d)
                a = lines[t(w+d)]
                b = lines[t(w-d)]
                if a in ops:
                    src = int_(b)
                    src_ = t(w-d)
                    dest = t(w+d)
                elif b in ops:
                    src = int_(a)
                    src_ = t(w+d)
                    dest = t(w-d)
                else:
                    pass

                if type(new[dest]) is str and new[dest] in ops:
                    s = lines[src_]
                    if type(s) is int or s.isnumeric():
                        new[dest] = initial[new[dest]]
                
                func = lines[dest]
                if lines[dest] in ops and type(new[dest]) is int:
                    if func == '+':
                        new[dest] += src
                    elif func == '*':
                        new[dest] *= src
                    elif func == '$':
                        new[dest] += src
                
                
                
        lines = new.copy().astype(str)
            
        printout = '\n'.join(''.join(str(s) for s in r) for r in new)
        print(printout,'\n')
                    
execute(sample, i=5)

(9, 23)
$ 6
* 5
5 6
30 2
$ 0
_______________________
_5_6-6_________________
_|/____________________
_60-2___________________
_|_____________________
_$_____________________
_______________________
_______________________
_______________________ 

$ 0
0 0
0 0
0 0
0 60
_______________________
_5_6-6_________________
_|/____________________
_60-2___________________
_|_____________________
_60_____________________
_______________________
_______________________
_______________________ 

60 60
60 60
60 60
60 60
60 60
_______________________
_5_6-6_________________
_|/____________________
_60-2___________________
_|_____________________
_60_____________________
_______________________
_______________________
_______________________ 

60 60
60 60
60 60
60 60
60 60
_______________________
_5_6-6_________________
_|/____________________
_60-2___________________
_|_____________________
_60_____________________
_______________________
_______________________
_______________________ 

60 60
60 60

In [21]:
(5, 3) + (2, 9)

(5, 3, 2, 9)